# 8-qubit experiments

In [1]:
import sys
sys.path.append('../src')

import numpy as np

from third_meeting.noisy_8_qubit import get_circuits_and_outputs

from utils import simulate_with_noise_3D
from utils import DistanceVisualizer
from utils import TransitionPointsVisualizer
print("All imports successful!")

All imports successful!


In [2]:
# circuits_and_outputs = get_circuits_and_outputs()
# names = circuits_and_outputs['names']
# true_circuits = circuits_and_outputs['true_circuits']
# true_dists = circuits_and_outputs['true_dists']
# mirrored_circuits = circuits_and_outputs['mirrored_circuits']
# true_outputs = circuits_and_outputs['true_outputs']

# noise_levels = np.linspace(0, 0.02, 21)

# # Create ranges for each segment
# ranges = [
#     np.arange(10, 1000, 20),      # Usually big std reduction occurs here so we want to most precision
#     np.arange(1000, 2000, 50),    # Usually completely converges in this range
#     np.arange(2000, 3001, 100)
# ]
# shot_counts = np.concatenate(ranges)

# Simulate:
1-12:  different n_local configurations [ry, rz]
13-18: different real_amplitudes configurations
19:23: random circuits with increasing depth

In [3]:
# # Simulate circuits 0 to 23 (there might be problems with circuit 12)
# indices = [22, 23]
# for circuit_index in indices:
#     mirrored_circuit = mirrored_circuits[circuit_index]
#     true_output = true_outputs[circuit_index]

#     true_circuit = true_circuits[circuit_index]
#     true_dist = true_dists[circuit_index]

#     hellinger_data = simulate_with_noise_3D(
#              true_circuit, true_dist, noise_levels=noise_levels, shot_counts=shot_counts, distance_type='hellinger'
#     )

#     hamming_data = []
#     for i in range(10):
#         print(f"Simulation run {i+1}/10 for circuit index {circuit_index}")
#         H = simulate_with_noise_3D(
#             mirrored_circuit, true_output, noise_levels=noise_levels, shot_counts=shot_counts, distance_type='hamming'
#         )
#         hamming_data.append(H)

#     np.savez_compressed(
#         f'../data/third_meeting/circuit_{circuit_index}_data.npz',
#         name=circuits_and_outputs['names'][circuit_index],
#         shots_array=shot_counts,
#         noise_intensities=noise_levels,
#         hamming_data=hamming_data,
#         hellinger_data=hellinger_data
#     )
    

In [4]:
# # Load all circuit data and put it into one file

# shots_dataset = []
# noise_dataset = []
# names_dataset = []
# hamming_dataset = []
# hellinger_dataset = []

# # Load circuits 0 to 23
# for circuit_index in range(0, 24):
#     loaded = np.load(f'../data/third_meeting/circuit_{circuit_index}_data.npz', allow_pickle=True)
#     shots_dataset.append(loaded['shots_array'])
#     noise_dataset.append(loaded['noise_intensities'])
#     names_dataset.append(loaded['name'].item())
#     hamming_dataset.append(loaded['hamming_data'])
#     hellinger_dataset.append(loaded['hellinger_data'])

# # Save all data into one file
# np.savez_compressed(
#     f'../data/third_meeting/all_circuits_data.npz',
#     names=names_dataset,
#     shots_dataset=shots_dataset,
#     noise_dataset=noise_dataset,
#     hamming_data=hamming_dataset,
#     hellinger_data=hellinger_dataset
# )

# Visualization

load data

In [5]:
# Load all circuits and visualize
loaded = np.load(f'../data/third_meeting/all_circuits_data.npz', allow_pickle=True)
names_dataset = loaded['names']
shots_dataset = loaded['shots_dataset']
noise_dataset = loaded['noise_dataset']
hamming_dataset = loaded['hamming_data']
hellinger_dataset = loaded['hellinger_data']

## I. Difficulties on perfect quantum computers
Lets have a look at our Hellinger distance when we have a perfect quantum computer (no erros).  
Different circuits have different output distributions and some distributions require more samples to be "spotted".

In [6]:
# vis1_indices = [21, 22, 23, 19]
# for i in vis1_indices:
#     visualizer = DistanceVisualizer(shots_dataset[i], noise_dataset[i], names_dataset[i], hamming_dataset[i], hellinger_dataset[i], 8)
#     visualizer.create_dashboard(0)

# Now let us formally define the task (on a perfect quantum computer):

In the case of a perfect quantum computer we just have an ideal distribution and we take increasing number of samples from it.  
Due to the curse of dimensionality lower number of samples naturally leads to "noisy" sampled distribution so our Hellinger is high.  
**Important note:** With no depolarizing error the Hellinger distance is mathematically proven to converge to 0 as we increase the number of shots.

The steep drop is the **bias reduction phase** where you learn the distribution's structure. (basically sample has been seen/has not been seen)  
The slow convergence is the **variance reduction phase** where you refine the probability estimates.

<div>
<img src="../data/third_meeting/images/bias_variance1.jpg" width="500"/>
</div>

What we are looking for is the so called **transition point** between these two, usually after this point the error-decay
stablizes to a 1/n-like rate.  
Teoretically, the transition point is determined by the distribution's complexity (dimensionality, smoothness, smallest probability mass).  

Some distributions that require lot of samples to estimate properly - Cauchy's distribution (next meeting with worked example on discrete Cauchy):  
<div>
<img src="../data/third_meeting/images/cauchy.png" width="500"/>
</div>

Now let's have a look at the output distributions of the above circuits (21, 22, 23, 19):  
Unfortunately i didn't save their outputs, only Hellinger and Hamming distances :(

## II. Noisy Quantum computers
It's mathematically proveable that a small error in sampling induces a non-zero lower bound on the Hellinger distance that does not vanish, even with infinitely many samples.  
However, having noise allows us to calculate Hamming distance which may potentially help us determine the curvature of our Hellinger.  
We must take a look how the noise affects the transition point, because we need to use noise in order to get Hamming curves.  
If the transition point of Hellinger is larely invariant to noise and If the same holds for our Hamming(or it's std) we can then start to look at how well-correlated they are.  

In [7]:
# Visualize transition points of Hellinger of all circuits with graphs stacked under each other
visualizer2 = TransitionPointsVisualizer(names_dataset, shots_dataset, noise_dataset, hamming_dataset, hellinger_dataset, 8)
# visualizer2.hellinger_dashboard()

/home/kgergov/TUM/Masters-Thesis/notebooks/../src/utils.py:427: OptimizeWarning: Covariance of the parameters could not be estimated
  params_opt, params_cov = curve_fit(


In [8]:
# Example graph of circuit with both points marked
visualizer2.transition_points_dashboard(tolerance=0.05)

interactive(children=(Dropdown(description='Circuit:', options=((np.str_('QFT'), 0), (np.str_('varQC_linear_2r…

In [ ]:
# Let's artificially smooth the data and look at it again
from scipy.signal import savgol_filter

# Apply Savitzky-Golay filter twice before visualizing for stronger smoothing
smoothed_hellinger = [savgol_filter(hellinger_data, 11, 2) for hellinger_data in hellinger_dataset]

# Smooth Hamming dataset with savgol filter as well
smoothed_hamming = [savgol_filter(hamming_data, 11, 2) for hamming_data in hamming_dataset]

visualizer3 = TransitionPointsVisualizer(names_dataset, shots_dataset, noise_dataset, smoothed_hamming, smoothed_hellinger, 8)
visualizer3.transition_points_dashboard(tolerance=0.0005)

interactive(children=(Dropdown(description='Circuit:', options=((np.str_('QFT'), 0), (np.str_('varQC_linear_2r…

Possible next steps: finely-grained simulations in the 0-3000 shots range on more powerful computer for verification.
Number of hamming experiments should also be increased for smoother std 